In [1]:
import pandas as pd 
from pandas import Series, DataFrame
import numpy as np
#import altair as alt
!pip3 install PyPDF2
!pip3 install tabula-py
import PyPDF2
import locale
from tabula import read_pdf
import re
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

'en_US.UTF-8'

In [2]:
def fetchArmsRegisteredState(year,yearPageDict):
    pdfFileObj = open('data/firearms-commerce-'+str(year)+'.pdf','rb')
    # pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # a page object
    pageObj = pdfReader.getPage(yearPageDict[year])
    # extracting text from page.
    # this will print the text you can also save that into String
    page24 = pageObj.extractText()
    page24=page24[page24.find("Alabama"):]
    page24=page24[:page24.find("Total")]
    
    page24=page24.replace("US \nTerritories \n","US Territories ")
    page24=page24.replace("of \nColumbia","of Columbia")
    page24=page24.replace("\n","")
    miscState = {'North Carolina':'North_Carolina','South Carolina':'South_Carolina','North Dakota':'North_Dakota',
                'South Dakota':'South_Dakota','New Hampshire':'New_Hampshire','New Jersey':'New_Jersey',
                'New Mexico':'New_Mexico','Rhode Island':'Rhode_Island','West Virginia':'West_Virginia',
                'New York':'New_York','Other US Territories':'Other_US_Territories','District of Columbia':'District_of_Columbia'
                }
    for key in miscState.keys():
            page24=page24.replace(key, miscState[key])
    
    page24=re.sub(r' ([A-Z])', r'\n\1', page24)
    #print(page24)
    page1 = page24.split('\n')
    strip_list = []

    for a in page1:
        stripS = a.rstrip()
        strip_list.append(stripS)
    pdf_df = pd.DataFrame([sub.rstrip().replace(',','').split(" ") for sub in strip_list])
    pdf_df.columns =['state','weapons','devices','machinegun','silencer','short_barreled_rifle','short_barreled_gun','Total']
    pdf_df=pdf_df[pdf_df.weapons.notnull()]


    #pdf_df=pdf_df.iloc[:,:].machinegun.replace(',', '').astype(float)

    pdf_df = pdf_df.astype({"state": str,"weapons":int,"devices":int, "machinegun": int,"silencer":int,"short_barreled_rifle":int,"short_barreled_gun":int,"Total":int})

    pdf_df['Total']=pdf_df.weapons+pdf_df.machinegun+pdf_df.devices+pdf_df.silencer+pdf_df.short_barreled_rifle+pdf_df.short_barreled_gun
    pdf_df[pdf_df.state!='Total']
    pdf_df['Year']=year
    pdfFileObj.close()
    return pdf_df

In [3]:
yearPageDict={2010:24,2011:14,2012:14,2013:15,2014:15,2015:15,2016:15,2017:15}
# pdf file object
# you can find find the pdf file with complete code in below
df_final=pd.DataFrame()
for year in [2010,2011,2012,2013,2014,2016,2017]:
    #df=processOnly2015(year,yearPageDict)
    df=fetchArmsRegisteredState(year,yearPageDict)
    df_final = pd.concat([df_final,df])

df_final.to_csv("licensed_ownership_bystate.csv")
df_final

,state,weapons,devices,machinegun,silencer,short_barreled_rifle,short_barreled_gun,Total,Year
0,Alabama,1085,38742,14201,7195,791,2013,64027,2010
1,Alaska,307,2840,1602,1779,419,1071,8018,2010
2,Arkansas,568,43567,4188,5705,839,977,55844,2010
3,Arizona,1029,63380,15633,13828,4388,1747,100005,2010
4,California,3562,195423,26237,4798,2643,10103,242766,2010
5,Colorado,856,34346,5766,4792,1675,1348,48783,2010
6,Connecticut,633,8846,21943,4568,878,2518,39386,2010
7,District_of_Columbia,69,34059,4103,160,182,978,39551,2010
8,Delaware,32,1974,515,257,85,441,3304,2010
9,Florida,2906,92609,27607,22517,6019,5313,156971,2010
